In [1]:
import os
import numpy as np
from PIL import Image
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
device = 'cuda'
import open_clip

model_type = 'ViT-H-14'

vlmodel, preprocess_train, feature_extractor = open_clip.create_model_and_transforms(
    'ViT-H-14', pretrained = '/home/ubuntu/Downloads/open_clip_pytorch_model.bin', device = device
)

/home/ubuntu/.conda/envs/visual_decoding/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def load_ep_data(image_folderpath, vlmodel, preprocess_train, batch_size=20):    
    image_list = []
    image_label_list = []
    
    for image_dir in os.listdir(image_folderpath):
        image_label = image_dir.split(".")[0]
        image = Image.open(os.path.join(image_folderpath, f"{image_label}.jpg")).convert("L")
        image_label = int(image_label) - 1
        
        image_list.append(image)
        image_label_list.append(image_label)
    
    def image_encoder(images, vlmodel, preprocess, batch_size):
        features = []
        for i in range(0, len(images), batch_size):
            batch_images = images[i:i+batch_size]
            batch_input = torch.stack([preprocess(img) for img in batch_images]).to('cuda')
            with torch.no_grad():
                batch_features = vlmodel.encode_image(batch_input)
                batch_features /= batch_features.norm(dim=-1, keepdim=True)
            features.append(batch_features.cpu())
        return torch.cat(features)
    
    image_features = image_encoder(image_list, vlmodel, preprocess_train, batch_size)
    return image_list, image_label_list, image_features

In [17]:
image_list, image_label_list, image_features = load_ep_data("/media/ubuntu/sda/data/trigger/NaturalImages_new_2", vlmodel=vlmodel, preprocess_train=preprocess_train)

In [19]:
image_feature_list = {}
for i in range(len(image_label_list)):
    image_feature_list[image_label_list[i]] = image_features[i]

In [23]:
import pickle
with open("image_feature_list.pkl", 'wb') as f:
    pickle.dump(image_feature_list,f)